# Lecture 1: Getting Started
## Graphs & Sessions

    
### 1. Tensor
- def: n-dimensional matrix
    - 0-d tensor: scalar (number)
    - 1-d tensor: vector
    - 2-d tensor: matrix
- tensors are data -> data flow=tensor flow
    
### 2. Data flow graphs
- TF separates definition of computations from their execution
- phases
    1. assemble a graph
    2. use a session to execute operations in the graph
- elements
    1. nodes:  operators, variables, and constants
    2. edges:  tensors




In [2]:
import tensorflow as tf

In [5]:
a = tf.add(3, 5) #can be visualized by tensorboard
print( a )

Tensor("Add:0", shape=(), dtype=int32)


### 3. Get the value of a
- Create a session, assign it to a variable (sess)
- Within the session, evaluate the graph to fetch the value of a

In [10]:
sess = tf.Session() #get the value of a
print(sess.run(a))
sess.close() 

# with clause takes care
# of sess.close()
with tf.Session() as sess:
    print(sess.run(a))

8
8


In [13]:
x = 2
y = 3
op1 = tf.add(x, y)
op2 = tf.multiply(x, y)
useless = tf.multiply(x, op1)
op3 = tf.pow(op2, op1)
with tf.Session() as sess:
    op3 = sess.run(op3) # session won’t compute values of useless


In [ ]:
with tf.Session() as sess:
    op3, not_useless = sess.run([op3, useless])

Possible to break graphs into several chunks and run them parallelly across multiple CPUs, GPUs, or devices

In [ ]:
# Creates a graph.
with tf.device('/gpu:2'):
 a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='a')
 b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='b')
 c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print sess.run(c)

### 4. more than 1 graph?
- not necessary
- Multiple graphs require multiple sessions, each will try to use all available resources by default
- Can't pass data between them without passing them through python/numpy, which doesn't work in distributed 
- better to have <u>disconnected subgraphs within one graph</u>

### 5. Create a graph

To add operators to a graph, set it as default:

In [ ]:
g = tf.Graph()
with g.as_default():
    a = 3
    b = 5
    x = tf.add(a, b)
sess = tf.Session(graph=g)
with tf.Session() as sess:
    sess.run(x)

To handle the default graph:

In [ ]:
g = tf.get_default_graph()

do NOT mix default graph and user created graphs

- prone to errors:

In [ ]:
g = tf.Graph()
# add ops to the default graph
a = tf.constant(3)
# add ops to the user created graph
with g.as_default():
b = tf.constant(5)

- improvement, but not good enough because no more than one graph:

In [ ]:
g1 = tf.get_default_graph()
g2 = tf.Graph()
# add ops to the default graph
with g1.as_default():
a = tf.Constant(3)
# add ops to the user created graph
with g2.as_default():
b = tf.Constant(5)

### 6. Why graphs?
1. <u>Save computation</u> (only run subgraphs that lead to the values you want to fetch)
2. Break computation into small, differential pieces to <u>facilitates auto-differentiation</u>
3. <u>Facilitate distributed computation</u>, spread the work across multiple CPUs, GPUs, or devices
4. Many common machine learning models are commonly taught and visualized as directed graphs already